In [1]:
#!/bin/sh

'''In this workbook, I will be querying data from the NYC OpenData API and storing it into a MySQL database. 
    I will create both a time-invariant (store info) and time-varying (inspection info) database. '''

import requests
import json

# Connect to the Citibike API and fetch the data
def getOpenData():
    ''' 
    Connects to the NYC OpenData Restaurant Inspection Results API, and returns back a list
    of dictionaries, each dictionary corresponding
    to an inspection instance
    '''
    # Let's get the data from the Citibike API
    url = 'https://data.cityofnewyork.us/resource/9w7m-hzhe.json?$$app_token=yewO3Q85FrsdtwG6azAWeMPol'
    resp = requests.get(url)
    results = json.loads(resp.text)
    return results

store_data = getOpenData()
store_data


[{u'action': u'Violations were cited in the following area(s).',
  u'boro': u'STATEN ISLAND',
  u'building': u'1660      ',
  u'camis': u'40395935',
  u'critical_flag': u'Not Critical',
  u'cuisine_description': u'Hamburgers',
  u'dba': u"MCDONALD'S",
  u'grade': u'A',
  u'grade_date': u'2015-05-12T00:00:00.000',
  u'inspection_date': u'2015-05-12T00:00:00.000',
  u'inspection_type': u'Cycle Inspection / Initial Inspection',
  u'phone': u'7184943400',
  u'record_date': u'2016-02-29T06:01:46.000',
  u'score': u'12',
  u'street': u'RICHMOND AVENUE                                   ',
  u'violation_code': u'10F',
  u'violation_description': u'Non-food contact surface improperly constructed. Unacceptable material used. Non-food contact surface or equipment improperly maintained and/or not properly sealed, raised, spaced or movable to allow accessibility for cleaning on all sides, above and underneath the unit.',
  u'zipcode': u'10314'},
 {u'action': u'Violations were cited in the following

In [2]:
import MySQLdb as mdb
import sys

# Setup the database in which we will store the inspection data
def connectDB():
    con = mdb.connect(host = 'localhost', 
                      user = 'root', 
                      passwd = 'dwdstudent2015', 
                      charset = 'utf8', use_unicode=True);
    return con

In [3]:
def createOpenDB(con, db_name):
    ''' 
    Connects to the database and creates (if it does not exist)
    the database and a time varying and time-invariant table.
    '''
    # Query to create a database
    create_db_query = "CREATE DATABASE IF NOT EXISTS {0} DEFAULT CHARACTER SET 'utf8'".format(db_name)

    # Create a database
    cursor = con.cursor()
    cursor.execute(create_db_query)
    cursor.close()
    pass

    '''This table stores information about the restaurant'''

def createTimeInvariantTable(con, db_name, table_name):
    cursor = con.cursor()
    # Create a table
    # The {0} and {1} are placeholders for the parameters in the format(....) statement
    create_table_query = '''CREATE TABLE IF NOT EXISTS {0}.{1} 
                                    (store_id char(15),
                                    store_name varchar(255), 
                                    cuisine_type varchar(255),
                                    borough varchar(255),
                                    phone varchar(255), 
                                    building_number varchar(255),
                                    street varchar(255),
                                    zipcode varchar(255),
                                    PRIMARY KEY(store_id)
                                    )'''.format(db_name, table_name)
    cursor.execute(create_table_query)
    cursor.close()
    
    '''This table stores information about the inspection'''
    
def createTimeVaryingTable(con, db_name, table_name):
    cursor = con.cursor()
    # Create a table
    # The {0} and {1} are placeholders for the parameters in the format(....) statement
    create_table_query = '''CREATE TABLE IF NOT EXISTS {0}.{1} 
                                    (store_id char(25),
                                    store_name varchar(250),
                                    inspection_dt datetime,
                                    record_dt datetime,
                                    inspection_type varchar(250),
                                    violation_code varchar(250),
                                    grade varchar(20),
                                    PRIMARY KEY(store_id, inspection_dt, violation_code)
                                    )'''.format(db_name, table_name)
    cursor.execute(create_table_query)
    cursor.close()

con = connectDB()
db_name = 'OpenData'
createOpenDB(con, db_name)
station_table = 'store_info'
createTimeInvariantTable(con, db_name, station_table)
station_table = 'inspection_info'
createTimeVaryingTable(con, db_name, station_table)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:11: Warning: Can't create database 'OpenData'; database exists
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:32: Warning: Table 'store_info' already exists
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:51: Warning: Table 'inspection_info' already exists


In [4]:
# Go over the inspection data and store the time-invariant data into the time-invariant data table
def insertStation(con, db_name, table_name, store_id, store_name, cuisine, borough, phone, building_number, street, zipcode):
    db_name = 'OpenData'
    table_name = 'store_info'
    
    query_template = '''INSERT IGNORE INTO {0}.{1}(store_id,
                                    store_name, 
                                    cuisine_type,
                                    borough,
                                    phone, 
                                    building_number,
                                    street,
                                    zipcode)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)'''.format(db_name, table_name)

    cursor = con.cursor()
    query_parameters = (store_id, store_name, cuisine, borough, phone, building_number, street, zipcode)
    cursor.execute(query_template, query_parameters)
    cursor.close()
    return

def storeTimeInvariantData(con, open_data):

    db_name = 'OpenData'
    table_name = 'store_info'
    
    for x in open_data:
        store_id = x["camis"]
        try:
            store_name = x["dba"]
        except KeyError:
            pass
        cuisine = x["cuisine_description"]
        borough = x["boro"]
        phone = x["phone"]
        building_number = x["building"]
        street = x["street"]
        zipcode = x["zipcode"]
        insertStation(con, db_name, table_name, store_id, store_name, cuisine, borough, phone, building_number, street, zipcode)
        
        print "Inserted", store_name
    # Writes the data in the database, for sure
    con.commit()
    return
 


storeTimeInvariantData(con, store_data)

Inserted MCDONALD'S
Inserted EASTERN BLOC
Inserted WESTSIDE BREWHOUSE
Inserted CAS' WEST INDIAN & AMERICAN RESTAURANT
Inserted THE RUM HOUSE
Inserted HAPPY STAR BAKERY
Inserted LA MEDITERRANEE RESTAURANT
Inserted YONG SHENG RESTAURANT
Inserted VERLAINE
Inserted EDDIE'S DELI & PIZZA
Inserted THE KENT ALE HOUSE
Inserted GREENSTREETS SALADS
Inserted KIKOO SUSHI
Inserted HECHO EN DUMBO
Inserted RIVERDALE STEAK HOUSE
Inserted WO HOP CITY
Inserted DAE DONG MANOR
Inserted SMORGAS CHEF/ CREPES DU NORD
Inserted LIONS BEER STORE
Inserted CAFE SHIRIN
Inserted FORT HAMILTON DINER
Inserted GLATT ALA CARTE
Inserted CAESAR'S PALACE
Inserted EL TAPATIO MEXICAN RESTAURANT
Inserted TROPICAL 128
Inserted LUNA
Inserted SHINOBI RAMEN
Inserted AKI SUSHI
Inserted EL RICO TINTO BAKERY
Inserted TASTY FAST FOOD
Inserted ROSE BENGAL
Inserted BONAO BAR AND GRILL
Inserted SYCAMORE
Inserted SHARMEL CATERERS
Inserted JUNIPER
Inserted MIGHTY QUINN'S BBQ
Inserted SUGAZ BAR
Inserted OLI 99¢ FRESH PIZZA
Inserted MAMA'S 

In [5]:
from datetime import datetime

# Go over the inspection data and store the time-varying
# data into the appropriate table in the database

def storeTimeVaryingData(con, open_data):
    '''
    Accepts as a parameter a list of dictionaries, where
    each dictionary is an inspection.
    Goes over these dictionaries, and stores in the database
    the entries that are time variant.
    The data is stored if the restaurant ID, inspection date, and violation code, represent a unique 3-pair.
    
    Because not every instance has a violation code or grade, we use a try/except statement
    '''
    
    db_name = 'OpenData'
    table_name = 'inspection_info'
    count = 0
    for store in open_data:
        store_id = store["camis"]
        try:
            store_name = store["dba"]
        except KeyError:
            pass
        try:
            inspection_date  = store["inspection_date"]
        except KeyError:
            pass
        record_date  = store["record_date"]
        try:
            violation_code = store["violation_code"]
        except KeyError:
            pass
        try:
            grade = store["grade"]
        except KeyError:
            grade = None
            pass
        try:
            inspection_type = store["inspection_type"]
        except KeyError:
            pass
        insertStatus(con, db_name, table_name, store_id, store_name, inspection_date, record_date, inspection_type, violation_code, grade)
        count += 1
        print "Inserted", store_name
        
    con.commit()
    
    return
        
def insertStatus(con, db_name, table_name, 
                  store_id, store_name, inspection_date, record_date, inspection_type, violation_code, grade):
    query_template = '''INSERT IGNORE INTO {0}.{1}(store_id,
                                    store_name,
                                    inspection_dt,
                                    record_dt,
                                    inspection_type,
                                    violation_code,
                                    grade)
                VALUES (%s, %s, %s, %s, %s, %s, %s)'''.format(db_name, table_name)

    cursor = con.cursor()
    query_parameters = (store_id, store_name, inspection_date, record_date, inspection_type, violation_code, grade)
    cursor.execute(query_template, query_parameters)
    cursor.close()

    return


storeTimeVaryingData(con, store_data)

Inserted MCDONALD'S
Inserted EASTERN BLOC
Inserted WESTSIDE BREWHOUSE
Inserted CAS' WEST INDIAN & AMERICAN RESTAURANT
Inserted THE RUM HOUSE
Inserted HAPPY STAR BAKERY
Inserted LA MEDITERRANEE RESTAURANT
Inserted YONG SHENG RESTAURANT
Inserted VERLAINE
Inserted EDDIE'S DELI & PIZZA
Inserted THE KENT ALE HOUSE
Inserted GREENSTREETS SALADS
Inserted KIKOO SUSHI
Inserted HECHO EN DUMBO
Inserted RIVERDALE STEAK HOUSE
Inserted WO HOP CITY
Inserted DAE DONG MANOR
Inserted SMORGAS CHEF/ CREPES DU NORD
Inserted LIONS BEER STORE
Inserted CAFE SHIRIN
Inserted FORT HAMILTON DINER
Inserted GLATT ALA CARTE
Inserted CAESAR'S PALACE
Inserted EL TAPATIO MEXICAN RESTAURANT
Inserted TROPICAL 128
Inserted LUNA
Inserted SHINOBI RAMEN
Inserted AKI SUSHI
Inserted EL RICO TINTO BAKERY
Inserted TASTY FAST FOOD
Inserted ROSE BENGAL
Inserted BONAO BAR AND GRILL
Inserted SYCAMORE
Inserted SHARMEL CATERERS
Inserted JUNIPER
Inserted MIGHTY QUINN'S BBQ
Inserted SUGAZ BAR
Inserted OLI 99¢ FRESH PIZZA
Inserted MAMA'S 